In [4]:
#https://stackoverflow.com/a/54594071
from IPython.display import HTML

HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Display code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Display code"></form>''')


In [5]:
import warnings
warnings.filterwarnings('ignore')
import interface                     # loads the user friendly GUI
interface.makeScrapeInterface()

# Floriano

Text(value='', description='Text 1:')

ToggleButtons(description='Operator', options=('and', 'or', 'not'), value='and')

Text(value='', description='Text 2:')

Text(value='', description='EDCS-ID:')

Text(value='', description='Publication:')

...

Button(description='Inscription Genus...', style=ButtonStyle())

SelectMultiple(description='inscription genus / personal status:', layout=Layout(display='none'), options=(Non…

SelectMultiple(description='and not:', layout=Layout(display='none'), options=(None, 'Augusti/Augustae', 'carm…

Button(description='Scrape!', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

In [6]:
import map_interface 
map_interface.make_map_interface()



Text(value='', description='Map Title:')

Button(description='Generate New Maps!', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

In [ ]:
HTML('''<script>

</script>
<form action="javascript:IPython.notebook.execute_cells_below()"><input type="submit" id="toggleButton" value="Refresh"></form>''')

In [ ]:
import geopandas
import matplotlib.pyplot as plt
from frictionless import extract
from pathlib import Path
from pprint import pprint
import pandas
import os
import subprocess
import geoplot
import matplotlib
import shutil
import glob
# https://geopandas.org/reference/geopandas.GeoDataFrame.html#geopandas.GeoDataFrame
from shapely.geometry import Point
# https://geopandas.org/gallery/plotting_basemap_background.html#sphx-glr-gallery-plotting-basemap-background-py
import contextily as ctx


DATA_DIR = "output"
SUPPORTING_DATA = Path("awmc.unc.edu")
SUPPORTING_DATA = SUPPORTING_DATA / "awmc" / "map_data" / "shapefiles"
ROMAN_ROADS_SHP = SUPPORTING_DATA / "ba_roads" / "ba_roads.shp"
PROVINCES_SHP   = SUPPORTING_DATA / "cultural_data" / "political_shading" / "roman_empire_ad_117" / "shape" / "roman_empire_ad_117.shp"
CITIES_DATA     = Path("cities") / "Hanson2016_Cities_OxREP.csv"

cities_rows = extract(CITIES_DATA)
cities_dataframe = pandas.DataFrame(cities_rows)


#https://cmdlinetips.com/2018/02/how-to-subset-pandas-dataframe-based-on-values-of-a-column/

roads_4326 = geopandas.read_file(ROMAN_ROADS_SHP).to_crs(epsg=4326)
provinces_4326 = geopandas.read_file(PROVINCES_SHP).to_crs(epsg=4326)


cities_geodataframe_4326 = geopandas.GeoDataFrame(
cities_dataframe,
geometry=geopandas.points_from_xy(
  cities_dataframe["Longitude (X)"],
  cities_dataframe["Latitude (Y)"]),
crs="EPSG:4326").to_crs(epsg=4326)
cities_geodataframe_4326.drop("Longitude (X)", inplace=True, axis=1)
cities_geodataframe_4326.drop("Latitude (Y)", inplace=True, axis=1)

#pprint(cities_geodataframe_4326)
geopandas.options.use_pygeos = True

In [ ]:
import folium
from folium import plugins
import random
import simplejson as json
import textwrap
import tldextract
import randomcolor

# TODO https://nbviewer.jupyter.org/github/python-visualization/folium_contrib/blob/master/notebooks/HereMapsApiExplorer_no_creds.ipynb get this out of the display file


def makeDataframe(data_file, epsg=3857):
    
    # pprint(WMS_LAYERS)
    # pprint([op.name for op in wms.operations])

    # https://frictionlessdata.io/tooling/python/extracting-data/
    # Handles multiline columns cleanly.
    data_filename = os.path.basename(data_file)
    #print(f"Making {data_filename}\n\troads: {roads}\n\tprovinces: {provinces}\n\tcities: {cities}\n")
    import_rows = extract(data_file)
    
    #pprint(import_rows[0])
    import_dataframe = pandas.DataFrame(import_rows)

    #cities_3857 = geopandas.read_file(CITIES_SHP).to_crs(epsg=3857)

    point_geodataframe = geopandas.GeoDataFrame(
    import_dataframe[import_dataframe.Longitude.notnull()],
    geometry=geopandas.points_from_xy(
    import_dataframe[import_dataframe.Longitude.notnull()].Longitude,
    import_dataframe[import_dataframe.Longitude.notnull()].Latitude),
    crs="EPSG:4326")
    point_geodataframe[['Latitude', 'Longitude']] = point_geodataframe[['Latitude', 'Longitude']].apply(pandas.to_numeric)
    
    def linkify(link_field):
        if not link_field:
            return None
        if " | " in link_field:
            links = ['']
            for link in link_field.split(" | "):
                
                link = f"<a href='{link}' target='_blank'>{tldextract.extract(link).registered_domain}</a>"
                links.append(link)
            return f"<ul>{'<li>'.join(links)}</ul>"
        return f"<ul><li><a href='{link_field}' target='_blank'>{tldextract.extract(link_field).registered_domain}</a></li></ul>"

    point_geodataframe['Links'] = point_geodataframe['Links'].apply(linkify)

    point_geodataframe['inscription'] = point_geodataframe['inscription'].apply(lambda x: textwrap.shorten(x, width=255))
    
    point_geodataframe_3857 = point_geodataframe.to_crs(epsg=epsg)
    return point_geodataframe_3857

output_maps ={}

xmin, ymin, xmax, ymax = (0,0,10000,10000)


for file in glob.glob(f"{DATA_DIR}/*.tsv"):
    df =  makeDataframe(file, epsg=4326)
    #pprint(df)
    map_xmin, map_ymin, map_xmax, map_ymax = df.total_bounds

    xmin = max(xmin, map_xmin)
    ymin = max(ymin, map_ymin)
    xmax = min(xmax, map_xmax)
    ymax = min(ymax, map_ymax)
    
    output_maps[file] = df.to_json()



    
# Create a map using the Map() function and the coordinates for Boulder, CO
scrape_map = folium.Map(location=[41.9028, 12.4964], 
                        zoom_start=5,
                        
                        #tiles='https://{s}.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}{r}.png',
                        #attr = '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>',
                        
                        tiles = 'https://stamen-tiles-{s}.a.ssl.fastly.net/terrain-background/{z}/{x}/{y}.png',
                       	attr = 'Map tiles by <a href="http://stamen.com">Stamen Design</a>, <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a> &mdash; Map data &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
)

# Display the map

scrape_map.fit_bounds([[ymin,xmin], [ymax,xmax]])

#cities = folium.features.GeoJson(cities_geodataframe_4326.to_json()).add_to(scrape_map)
# Running the above crashes the tab

# scrape_map.add_child(cities)
#https://leafletjs.com/reference-1.6.0.html#path-option
road_style = {'color':'#000000',
              'opacity': 0.5}
roads = folium.features.GeoJson(roads_4326.to_json(),
                                name='Roads',
                                style_function=lambda x: road_style).add_to(scrape_map)




#https://stackoverflow.com/a/45081821
r = lambda: (random.randint(0,255))


rand_color = randomcolor.RandomColor()
def province_style(arg):
    return {
        'weight': 2,
        'opacity': 1,
        'color': '#000000',
        'dashArray': 3,
        'fillOpacity': 0.4,
        'fillColor':  rand_color.generate(format_="hex", luminosity="dark"),
    }
provinces = folium.features.GeoJson(provinces_4326.to_json(), name='Province', style_function=province_style).add_to(scrape_map)



def point_style(arg, map_name):
    #print(map_name)
    return {
        'radius':5,
        'weight': 4,
        'opacity': 1,
        'color': randomcolor.RandomColor(map_name).generate(format_="hex", luminosity="bright"),
        'lineColor':"#000000",
        'dashArray': 3,
        'fillOpacity': 0.7,
        'fillColor': randomcolor.RandomColor(map_name).generate(format_="hex", luminosity="bright"),
    }
for map_json in output_maps:
    # https://github.com/python-visualization/folium/issues/1385
    print(map_json)
    folium.features.GeoJson(output_maps[map_json],
                            name=f"{map_json}",
                            style_function=lambda x: point_style(x,map_json),
                            marker=folium.CircleMarker(),
                            popup=folium.GeoJsonPopup(["EDCS-ID","Links","inscription"])
                           ).add_to(scrape_map)

folium.LayerControl().add_to(scrape_map)
scrape_map